[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pinecone-io/examples/blob/master/docs/pinecone-quickstart.ipynb) [![Open nbviewer](https://raw.githubusercontent.com/pinecone-io/examples/master/assets/nbviewer-shield.svg)](https://nbviewer.org/github/pinecone-io/examples/blob/master/docs/pinecone-quickstart.ipynb)

# Pinecone Database quickstart

This notebook shows you how to set up and use Pinecone Database for high-performance similarity search.

## Install an SDK

Pinecone provides [SDKs](https://docs.pinecone.io/reference/pinecone-sdks) in multiple languages.

For this quickstart, install the Python SDK and a library that makes it easy to sign up with Pinecone:

In [ ]:
!pip install -qU \
    pinecone \
    pinecone-notebooks

## Get an API key

You need an API key to make calls to your Pinecone project.

Use the widget below to generate a key. If you don't have a Pinecone account, the widget will sign you up for the free Starter plan.

In [2]:
import os

if not os.environ.get("PINECONE_API_KEY"):
    from pinecone_notebooks.colab import Authenticate
    Authenticate()

## Initialize a client

Use the generated API key to intialize a client connection to Pinecone:

In [3]:
from pinecone import Pinecone, ServerlessSpec

api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=api_key)

## Generate vectors

A [vector embedding](https://www.pinecone.io/learn/vector-embeddings/) is a numerical representation of data that enables similarity-based search in vector databases like Pinecone. To convert data into this format, you use an embedding model.

For this quickstart, use the [`multilingual-e5-large`](https://docs.pinecone.io/models/multilingual-e5-large) embedding model hosted by Pinecone to [convert](https://docs.pinecone.io/guides/inference/generate-embeddings) four sentences about apples into vectors, three concerning their health benefits, one concerning their cultivation.

In [ ]:
# Define a sample dataset where each item has a unique ID, text, and category
data = [
    {
        "id": "rec1",
        "text": "Apples are a great source of dietary fiber, which supports digestion and helps maintain a healthy gut.",
        "category": "digestive system"
    },
    {
        "id": "rec2",
        "text": "Apples originated in Central Asia and have been cultivated for thousands of years, with over 7,500 varieties available today.",
        "category": "cultivation"
    },
    {
        "id": "rec3",
        "text": "Rich in vitamin C and other antioxidants, apples contribute to immune health and may reduce the risk of chronic diseases.",
        "category": "immune system"
    },
    {
        "id": "rec4",
        "text": "The high fiber content in apples can also help regulate blood sugar levels, making them a favorable snack for people with diabetes.",
        "category": "endocrine system"
    }
]

# Convert the text into numerical vectors that Pinecone can index
embeddings = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[d["text"] for d in data],
    parameters={
        "input_type": "passage",
        "truncate": "END"
    }
)

print(embeddings)

## Create an index

In Pinecone, you store data in an [index](https://docs.pinecone.io/guides/indexes/understanding-indexes).

Create a serverless index that matches the dimension (`1024`) and similarity metric (`cosine`) of the `multilingual-e5-large` model you used in the previous step, and choose a [cloud and region](https://docs.pinecone.io/guides/indexes/understanding-indexes#cloud-regions) for hosting the index:

In [5]:
index_name = "docs-quickstart-notebook"

In [6]:
import time

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud='aws',
            region='us-east-1'
        )
    )

# Wait for the index to be ready
while not pc.describe_index(index_name).status['ready']:
    time.sleep(1)

## Upsert vectors

Target your index and use the [`upsert`](https://docs.pinecone.io/guides/data/upsert-data) operation to load your vector embeddings into a new namespace.

**Note:** [Namespaces](https://docs.pinecone.io/guides/get-started/key-features#namespaces) let you partition records within an index and are essential for [implementing multitenancy](https://docs.pinecone.io/guides/get-started/implement-multitenancy) when you need to isolate the data of each customer/user.


In [ ]:
# Target the index
# In production, target an index by its unique DNS host, not by its name
# See https://docs.pinecone.io/guides/data/target-an-index
index = pc.Index(index_name)

# Prepare the records for upsert
# Each contains an 'id', the vector 'values',
# and the original text and category as 'metadata'
records = []
for d, e in zip(data, embeddings):
    records.append({
        "id": d["id"],
        "values": e["values"],
        "metadata": {
            "source_text": d["text"],
            "category": d["category"]
        }
    })

# Upsert the records into the index
index.upsert(
    vectors=records,
    namespace="example-namespace"
)

**Note:** To load large amounts of data, [import from object storage](https://docs.pinecone.io/guides/data/understanding-imports) or [upsert in large batches](https://docs.pinecone.io/guides/data/upsert-data#upsert-records-in-batches).

## Check the index

Pinecone is eventually consistent, so there can be a delay before your upserted vectors are available to query. Use the [`describe_index_stats`](https://docs.pinecone.io/guides/data/data-freshness/check-data-freshness) operation to check if the current vector count matches the number of vectors you upserted:

In [ ]:
time.sleep(10)  # Wait for the upserted vectors to be indexed

print(index.describe_index_stats())

## Search the index

Now, let’s say you want to search your index for information about “Disease prevention”.

Use the the `multilingual-e5-large` model hosted by Pinecone *to* convert your query into a vector embedding, and then use the [`query`](https://docs.pinecone.io/guides/data/query-data) operation to search for the three vectors in the index that are most semantically similar to the query vector:

In [ ]:
# Define your query
query = "Disease prevention"

# Convert the query into a numerical vector that Pinecone can search with
query_embedding = pc.inference.embed(
    model="multilingual-e5-large",
    inputs=[query],
    parameters={
        "input_type": "query"
    }
)

# Search the index for the three most similar vectors
results = index.query(
    namespace="example-namespace",
    vector=query_embedding[0].values,
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(results)

Notice that the response includes only records about the health benefits of apples, not the cultivation of apple.

## Add reranking

You can increase the accuracy of your search by reranking results based on their relevance to the query.

Use the `rerank` operation and the `bge-reranker-v2-m3` reranking model hosted by Pinecone to rerank the values of the documents.source_text fields and return only the two most relevant documents:

In [ ]:
# Rerank the search results based on their relevance to the query
ranked_results = pc.inference.rerank(
    model="bge-reranker-v2-m3",
    query="Disease prevention",
    documents=[
        {"id": "rec3", "source_text": "Rich in vitamin C and other antioxidants, apples contribute to immune health and may reduce the risk of chronic diseases."},
        {"id": "rec1", "source_text": "Apples are a great source of dietary fiber, which supports digestion and helps maintain a healthy gut."},
        {"id": "rec4", "source_text": "The high fiber content in apples can also help regulate blood sugar levels, making them a favorable snack for people with diabetes."}
    ],
    top_n=2,
    rank_fields=["source_text"],
    return_documents=True,
    parameters={
        "truncate": "END"
    }
)

print(ranked_results)


Notice that the two returned records are the most relevant for the query, the first relating to reducing chronic diseases, the second relating to preventing diabetes.

## Add filtering

You can use a [metadata filter](https://docs.pinecone.io/guides/data/understanding-metadata) to limit your search to records matching a filter expression.

Your upserted records contain a `category` metadata field. Now use that field as a filter to search for records in the “digestive system” category:

In [ ]:
# Search the index with a metadata filter
filtered_results = index.query(
    namespace="example-namespace",
    vector=query_embedding.data[0].values,
    filter={
        "category": {"$eq": "digestive system"}
    },
    top_k=3,
    include_values=False,
    include_metadata=True
)

print(filtered_results)

Notice that the response includes only the one record in the “digestive system” category.

## Clean up

When you no longer need the `docs-quickstart-notebook` index, use the [`delete_index`](https://docs.pinecone.io/reference/api/control-plane/delete_index) operation to delete it:

In [13]:
pc.delete_index(index_name)

After you delete an index, you cannot use it again or recover it.